In [95]:
import json
import numpy as np
import pandas as pd

import re
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# gensim
from gensim import corpora, models, similarities, matutils
import gensim
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


import preprocessor as p # 
import string # use string.punctuation to clean out punctuation

from pprint import pprint

from datetime import datetime
import pickle



from tqdm import tqdm

pd.set_option('display.max_colwidth', -1)

In [37]:
with open("clean_tweets.pkl", 'rb') as picklefile: 
    tweetsDF = pickle.load(picklefile)

## Preprocessing

In [38]:
cleanDF = pd.DataFrame(columns = ['id','full_text'])

In [39]:
cleanDF['full_text'] = tweetsDF['full_text']

In [40]:
cleanDF.drop('id',axis = 1).head()

,full_text
0,population control will not reverse or mitigate climate change here is the logic and evidence populationcontrollaw
1,i see climate change deniers are out in force today due to the cold weather in europe they still haven t bothered to research the issue properly and are still hung up on the global warming meaning climatechangeisreal
2,convenient truth al gore the father of global warming suggest that we tax ranchers cows passing gas he owns mansion flies around in his personal jet and leaves a carbon footprint like a herd of elephants now the dems switch the name climate change what a hoax
3,and yes it s consistent with climate change warming at the poles weakens the jetstream which blows warm air from the west and allows colder air in from the east
4,can lee s l do something to safe the global warming and climate changes


In [72]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop += ['climate', 'change', 'global', 'warming', 'climatechange', 'globalwarming', 'caroline', 'comes']

## Word2Vec

In [93]:
texts = [[word for word in document.lower().split() if word not in stop]
         for document in cleanDF.full_text]

print (texts)

[['population', 'control', 'reverse', 'mitigate', 'logic', 'evidence', 'populationcontrollaw'], ['see', 'deniers', 'force', 'today', 'due', 'cold', 'weather', 'europe', 'still', 'bothered', 'research', 'issue', 'properly', 'still', 'hung', 'meaning', 'climatechangeisreal'], ['convenient', 'truth', 'al', 'gore', 'father', 'suggest', 'tax', 'ranchers', 'cows', 'passing', 'gas', 'owns', 'mansion', 'flies', 'around', 'personal', 'jet', 'leaves', 'carbon', 'footprint', 'like', 'herd', 'elephants', 'dems', 'switch', 'name', 'hoax'], ['yes', 'consistent', 'poles', 'weakens', 'jetstream', 'blows', 'warm', 'air', 'west', 'allows', 'colder', 'air', 'east'], ['lee', 'l', 'something', 'safe', 'changes'], ['john', 'ashton', 'trying', 'elain', 'mayhem', 'co', 'necessarily', 'simple', 'terms', 'allowing', 'fracking', 'help', 'reduce', 'catastrophic', 'breakdown', 'wonder', 'get', 'message', 'best', 'keep', 'nvda', 'roll', 'united', 'resistance'], ['might', 'seem', 'counterintuitive', 'plays', 'role',

In [96]:
model = gensim.models.Word2Vec(texts, size=100, window=2, min_count=1, workers=2,sg=1)

2018-03-03 17:41:27,007 : INFO : collecting all words and their counts
2018-03-03 17:41:27,013 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-03-03 17:41:27,072 : INFO : PROGRESS: at sentence #10000, processed 122284 words, keeping 7437 word types
2018-03-03 17:41:27,128 : INFO : collected 10189 word types from a corpus of 211402 raw words and 17579 sentences
2018-03-03 17:41:27,131 : INFO : Loading a fresh vocabulary
2018-03-03 17:41:27,163 : INFO : min_count=1 retains 10189 unique words (100% of original 10189, drops 0)
2018-03-03 17:41:27,165 : INFO : min_count=1 leaves 211402 word corpus (100% of original 211402, drops 0)
2018-03-03 17:41:27,198 : INFO : deleting the raw counts dictionary of 10189 items
2018-03-03 17:41:27,200 : INFO : sample=0.001 downsamples 64 most-common words
2018-03-03 17:41:27,202 : INFO : downsampling leaves estimated 198724 word corpus (94.0% of prior 211402)
2018-03-03 17:41:27,228 : INFO : estimated required memory for 10

In [98]:
model.most_similar('paris' ,topn=8)

/Users/emilygeller/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('achieving', 0.9629377722740173),
 ('involvement', 0.9531314969062805),
 ('accord', 0.95160973072052),
 ('despite', 0.9443439245223999),
 ('goals', 0.9425873756408691),
 ('written', 0.9281982183456421),
 ('agreement', 0.9266886115074158),
 ('ahead', 0.9266612529754639)]